In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)
Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [3]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

def variable_summaries(var, name,dorelu):
  with tf.name_scope("summaries"):
    mean = tf.reduce_mean(var)
    tf.scalar_summary('mean/' + name, mean)
    if dorelu==1:
        var = tf.nn.relu6(var)
        
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_sum(tf.square(var - mean)))
    tf.scalar_summary('sttdev/' + name, stddev)
    tf.scalar_summary('max/' + name, tf.reduce_max(var))
    tf.scalar_summary('min/' + name, tf.reduce_min(var))
    tf.scalar_summary('sparsity/'+ name, tf.nn.zero_fraction(var))
    tf.histogram_summary(name, var)
    
def variable_summaries_list(varDict):
  for name,var in varDict.items():
    with tf.name_scope("summaries"):
      mean = tf.reduce_mean(var)
      tf.scalar_summary(name + 'mean/', mean)
      tf.scalar_summary(name + 'sttdev/',  tf.sqrt(tf.reduce_sum(tf.square(var - mean))))
      tf.scalar_summary(name + 'max/', tf.reduce_max(var))
      tf.scalar_summary(name + 'min/', tf.reduce_min(var))
      tf.scalar_summary(name + 'sparsity/', tf.nn.zero_fraction(var))
      tf.histogram_summary(name, var)

def testAccuracy(arrayValue,stopthresh=0.5):
  #   print('V1, V2, Diff, Std = (%.03f, %.03f, %.03f, %.03f)' % 
  #         (arrayValue[-1], np.mean(arrayValue[-7:-2]),np.abs(arrayValue[-1]-np.mean(arrayValue[-7:-2])), np.std(arrayValue[-7:-2])))
  notChanging = np.abs(arrayValue[-1]-np.mean(arrayValue[-7:-2])) < stopthresh*np.std(arrayValue[-7:-2])
  slopeNegative = np.mean(np.gradient(arrayValue[-7:-1]))<0
  return notChanging|slopeNegative
  

In [ ]:
def exploreData(datasetDict, paramDict):
  doWrite = 0; # this isn't working
  tf.reset_default_graph()
  tf.Graph().as_default()
  graph = tf.Graph()
  learnRate = paramDict['learnRate']
  with graph.as_default():
    # With Priors
    # The datas
    
    tf_trainDataset = tf.constant(datasetDict['trainDataset'])
    tf_trainLabels = tf.constant(datasetDict['trainLabels'])
    train_subset = len(datasetDict['trainLabels'])

    tf_trainDatasetRed = tf.constant(datasetDict['trainDatasetRed'])
    tf_trainLabelsRed = tf.constant(datasetDict['trainLabelsRed'])
    tf_validDataset = tf.constant(datasetDict['validDataset'])
    tf_validLabels = tf.constant(datasetDict['validLabels'])
    tf_testDataset = tf.constant(datasetDict['testDataset'])
    tf_testLabels = tf.constant(datasetDict['testLabels'])

    # Variables.
    #with tf.name_scope('hidden') as scope:
    layer_name = 'layer1_wprior'
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]),name='weights')
    biases = tf.Variable(tf.zeros([num_labels]),name='biases')
    # All priors are given the same prior: 90%
    priors = tf.Variable(tf.ones([train_subset])*np.float(paramDict['priorInit']),name='priors')
    # Training computation.
    # Find the classification estimation 'z' value
    logits = tf.matmul(tf_trainDataset, weights) + biases
    # Calculate the cross-entropy loss. 
    loss1 = tf.reduce_mean(tf.mul(
              tf.nn.relu6(priors*6),tf.nn.softmax_cross_entropy_with_logits(logits, tf_trainLabels)
                                  )
                         )/tf.reduce_mean(tf.nn.relu6(priors*6))
    if doWrite:
      varDict1={layer_name+'weights': weights}
      varDict1[layer_name+'biases'] = biases
      varDict1[layer_name+'priors'] = tf.nn.relu6(priors*6)  
      variable_summaries_list(varDict1)
      tf.scalar_summary('loss/' + layer_name, loss1)
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer1 = tf.train.GradientDescentOptimizer(learnRate).minimize(loss1)
    # Predictions for the training, validation. 
    # Do not use the priors to perform classification!
    train_prediction1 = tf.nn.softmax(logits)
    valid_prediction1 = tf.nn.softmax(tf.matmul(tf_validDataset, weights) + biases)
    test_prediction1 = tf.nn.softmax(tf.matmul(tf_testDataset, weights) + biases)

    # WITHOUT PRIORS
    layer_name = 'layer1_normal'
    weights2 = tf.Variable(weights.initialized_value(),name='weights2')
    biases2 = tf.Variable(biases.initialized_value(),name='biases2')
    # Training computation.
    # Find the classification estimation 'z' value
    logits2 = tf.matmul(tf_trainDataset, weights2) + biases2
    # Calculate the cross-entropy loss. 
    loss2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits2, tf_trainLabels))
    if doWrite:
      varDict2={layer_name+'weights': weights2}
      varDict2[layer_name+'biases'] = biases2
      variable_summaries_list(varDict2)
      tf.scalar_summary('loss/' + layer_name, loss2)    
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer2 = tf.train.GradientDescentOptimizer(learnRate).minimize(loss2)
    # Predictions for the training, validation. 
    train_prediction2 = tf.nn.softmax(logits2)
    valid_prediction2 = tf.nn.softmax(tf.matmul(tf_validDataset, weights2) + biases2)
    # Predictions for the test data set. 
    test_prediction2 = tf.nn.softmax(tf.matmul(tf_testDataset, weights2) + biases2)                                    

    # WITHOUT PRIORS REDUCED DATA
    layer_name = 'layer1_normal_reducedDatas'
    weights3 = tf.Variable(weights.initialized_value(),name='weights3')
    biases3 = tf.Variable(biases.initialized_value(),name='biases3')
    # Training computation.
    # Find the classification estimation 'z' value
    logits3 = tf.matmul(tf_trainDatasetRed, weights3) + biases3
    # Calculate the cross-entropy loss. 
    loss3 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits3, tf_trainLabelsRed))
    if doWrite:
      varDict3={layer_name+'weights': weights3}
      varDict3[layer_name+'biases'] = biases3
      variable_summaries_list(varDict3)
      tf.scalar_summary('loss/' + layer_name, loss3)    
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer3 = tf.train.GradientDescentOptimizer(learnRate).minimize(loss3)
    # Predictions for the training, validation. 
    train_prediction3 = tf.nn.softmax(logits3)
    valid_prediction3 = tf.nn.softmax(tf.matmul(tf_validDataset, weights2) + biases3)
    # Predictions for the test data set. 
    test_prediction3 = tf.nn.softmax(tf.matmul(tf_testDataset, weights3) + biases3)                                    

  train_acc_priors = []
  valid_acc_priors = []
  train_acc_normal = []
  valid_acc_normal = []
  train_acc_normal_red = []
  valid_acc_normal_red = []
#   num_steps = 40000
  num_steps = 40000
  train1 = True
  train2 = True
  train3 = True
  with tf.Session(graph=graph) as session:
    # Initialize
    tf.initialize_all_variables().run()    
    summary_op = tf.merge_all_summaries()
    train_dir = 'tmp'
    summary_writer = tf.train.SummaryWriter(train_dir, session.graph)  
    print('Initialized')
    for step in range(num_steps):
      # Run the computations.
      if train1:
        # With priors
        _, lvl1, predictions1 = session.run([optimizer1, loss1, train_prediction1])
      if train2:
        # Without priors
        _, lvl2, predictions2 = session.run([optimizer2, loss2, train_prediction2])
      if train3:
        # Without priors
        _, lvl3, predictions3 = session.run([optimizer3, loss3, train_prediction3])

      if (step % 100 == 0):
        if doWrite:
          summary_str = session.run(summary_op)
          summary_writer.add_summary(summary_str, step)  

        acc1 = accuracy(predictions1, datasetDict['trainLabels'])
        acc2 =  accuracy(valid_prediction1.eval(), datasetDict['validLabels'])
        train_acc_priors.append(acc1)
        valid_acc_priors.append(acc2)
        print('With prior. Loss at step %d: %f . Training accuracy: %.1f%% . Validation accuracy: %.1f%%' % (step, lvl1 , acc1, acc2))

        acc1 = accuracy(predictions2, datasetDict['trainLabels'])
        acc2 =  accuracy(valid_prediction2.eval(), datasetDict['validLabels'])
        train_acc_normal.append(acc1)
        valid_acc_normal.append(acc2)
        print('Nope prior. Loss at step %d: %f . Training accuracy: %.1f%% . Validation accuracy: %.1f%%' % (step, lvl2 , acc1, acc2))

        acc1 = accuracy(predictions3, trainLabelsRed)
        acc2 =  accuracy(valid_prediction3.eval(), datasetDict['validLabels'])
        train_acc_normal_red.append(acc1)
        valid_acc_normal_red.append(acc2)
        print('Nope prior. Loss at step %d: %f . Training accuracy: %.1f%% . Validation accuracy: %.1f%%' % (step, lvl3 , acc1, acc2))
      stopthresh = 0.5;
      if step > 1000:
        if testAccuracy(train_acc_priors) and train1:
          train1 = False
          modelPriorLastStep = step
          print('No longer training w/ priors')
        if testAccuracy(train_acc_normal) and train2:
          train2 = False
          modelNormalLastStep = step
          print('No longer training w/ normal method')
        if testAccuracy(train_acc_normal_red) and train3:
          train3 = False
          modelNormalRedLastStep = step
          print('No longer training w/ normal + reduced data')
      if train1==False and train2==False and train3==False:
        print('Stopping Training at step ', step)
        break;
#       modelPriorLastStep = step
#       modelNormalLastStep = step
#       modelNormalRedLastStep = step
    tp1 = accuracy(test_prediction1.eval(), dDict['testLabels']);
    tp2 = accuracy(test_prediction2.eval(), dDict['testLabels']);
    tp3 = accuracy(test_prediction3.eval(), dDict['testLabels'])
    print('Test accuracy with prior: %.1f%%' % tp1)
    print('Test accuracy no prior: %.1f%%' % tp2)
    print('Test accuracy no prior on small data: %.1f%%' % tp3)
    accDict = {'trainAcc_priors': train_acc_priors, 'validAcc_prior': valid_acc_priors,
              'trainAcc_normal': train_acc_normal, 'validAcc_normal': valid_acc_normal,
              'trainAcc_normalRed': train_acc_normal_red, 'validAcc_normalRed': valid_acc_normal_red, 
              'testAcc_prior': tp1, 'testAcc_normal': tp2, 'testAcc_normalRed': tp3,
            'modelPriorLastStep': modelPriorLastStep, 'modelNormalLastStep': modelNormalLastStep,'modelNormalRedLastStep': modelNormalRedLastStep}
    
  return accDict



In [ ]:

dDict = dict()
dDict['validDataset'] = valid_dataset
dDict['validLabels'] = valid_labels
dDict['testDataset'] = test_dataset
dDict['testLabels'] = test_labels


train_subset = 10000
train_dataset2 = train_dataset[:train_subset, :]
train_labels2 = train_labels[:train_subset]
dDict['trainDataset'] = train_dataset2

numTest = 5;
fracBadList = np.linspace(0,.8, num=numTest) 
fracBadList = [0.9]
outList = []
pDict = dict()
for i, fracBad in enumerate(fracBadList):
  pDict['priorInit'] = 0.5#(1-fracBad)*.99
  pDict['learnRate'] = 0.2
  nBad = np.int(fracBad*train_labels2.shape[0])
  print('Mislabeling %0.1f of data or %i / %i examples' % (fracBad, nBad, train_subset))
  rangeNess = np.arange(train_labels2.shape[0])
  np.random.shuffle(rangeNess)
  randomBreak = rangeNess[:nBad]
  trainL = train_labels2
  for i, rb in enumerate(randomBreak):
    trainL[rb][:] = trainL[rb][np.random.permutation(10)];
  trainDatasetRed = np.delete(train_dataset2,randomBreak,0)
  trainLabelsRed = np.delete(train_labels2,randomBreak,0)

  dDict['trainLabels'] = trainL
  dDict['trainDatasetRed'] = trainDatasetRed
  dDict['trainLabelsRed'] = trainLabelsRed  
  ar = exploreData(dDict, pDict)
  outList.append(ar)
  with open('results_long_0.9.dat', 'wb') as outfile:
    pickle.dump(outList, outfile, protocol=pickle.HIGHEST_PROTOCOL)

Mislabeling 0.9 of data or 9000 / 10000 examples
Initialized
With prior. Loss at step 0: 16.537668 . Training accuracy: 10.0% . Validation accuracy: 7.8%
Nope prior. Loss at step 0: 16.537682 . Training accuracy: 10.0% . Validation accuracy: 7.8%
Nope prior. Loss at step 0: 17.777157 . Training accuracy: 7.5% . Validation accuracy: 7.8%
With prior. Loss at step 100: 7.154669 . Training accuracy: 11.3% . Validation accuracy: 15.8%
Nope prior. Loss at step 100: 7.379275 . Training accuracy: 11.2% . Validation accuracy: 15.9%
Nope prior. Loss at step 100: 2.616482 . Training accuracy: 67.8% . Validation accuracy: 15.8%
With prior. Loss at step 200: 5.715145 . Training accuracy: 11.7% . Validation accuracy: 17.0%
Nope prior. Loss at step 200: 6.008849 . Training accuracy: 11.7% . Validation accuracy: 16.9%
Nope prior. Loss at step 200: 1.724699 . Training accuracy: 74.8% . Validation accuracy: 17.0%
With prior. Loss at step 300: 4.959508 . Training accuracy: 12.1% . Validation accuracy: 17

In [ ]:
outList[0].items()
import pandas as pd

##Results with 60% removed after doing the full list!?
With prior. Loss at step 21200: 0.688023 . Training accuracy: 40.0% . Validation accuracy: 11.1%
Nope prior. Loss at step 21200: 1.960189 . Training accuracy: 30.4% . Validation accuracy: 11.2%
Nope prior. Loss at step 21200: 1.319424 . Training accuracy: 56.7% . Validation accuracy: 10.8%
No longer training w/ normal + reduced data
Stopping Training at step  21200
Test accuracy with prior: 10.8%
Test accuracy no prior: 11.3%
Test accuracy no prior on small data: 11.5%
#When I reset all code this is what I get
With prior. Loss at step 14600: 0.124448 . Training accuracy: 44.9% . Validation accuracy: 67.4%
Nope prior. Loss at step 14600: 1.687886 . Training accuracy: 45.2% . Validation accuracy: 56.6%
Nope prior. Loss at step 14600: 0.065873 . Training accuracy: 99.5% . Validation accuracy: 24.2%
No longer training w/ normal method
Stopping Training at step  14600
Test accuracy with prior: 74.8%
Test accuracy no prior: 63.2%
Test accuracy no prior on small data: 81.1%

What the F!?

In [ ]:
with open('test.dat', 'wb') as outfile:
    pickle.dump(outList, outfile, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
'''
fracBad = 0.2;
nBad = np.int(fracBad*train_labels.shape[0])
rangeNess = np.arange(train_labels.shape[0])
np.random.shuffle(rangeNess)
randomBreak = rangeNess[:nBad]

for i, rb in enumerate(randomBreak):
    train_labels[rb][:] = train_labels[rb][np.random.permutation(10)];
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

doWrite = 0;
graph = tf.Graph()

with graph.as_default():
  # With Priors
  # The datas
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  #with tf.name_scope('hidden') as scope:
  layer_name = 'layer1_wprior'
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]),name='weights')
  biases = tf.Variable(tf.zeros([num_labels]),name='biases')
  # All priors are given the same prior: 90%
  priors = tf.Variable(tf.ones([train_subset])*0.9,name='priors')
  # Training computation.
  # Find the classification estimation 'z' value
  logits = tf.matmul(tf_train_dataset, weights) + biases
  # Calculate the cross-entropy loss. 
  loss1 = tf.reduce_mean(tf.mul(
            tf.nn.relu6(priors*6),tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
                                )
                       )/tf.reduce_mean(tf.nn.relu6(priors*6))
  if doWrite:
    varDict1={layer_name+'weights': weights}
    varDict1[layer_name+'biases'] = biases
    varDict1[layer_name+'priors'] = tf.nn.relu6(priors*6)  
    variable_summaries_list(varDict)
    tf.scalar_summary('loss/' + layer_name, loss)
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer1 = tf.train.GradientDescentOptimizer(0.5).minimize(loss1)
  # Predictions for the training, validation. 
  # Do not use the priors to perform classification!
  train_prediction1 = tf.nn.softmax(logits)
  valid_prediction1 = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction1 = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
                                   
  # WITHOUT PRIORS
  layer_name = 'layer1_normal'
  weights2 = tf.Variable(weights.initialized_value(),name='weights2')
  biases2 = tf.Variable(biases.initialized_value(),name='biases2')
  # Training computation.
  # Find the classification estimation 'z' value
  logits2 = tf.matmul(tf_train_dataset, weights2) + biases2
  # Calculate the cross-entropy loss. 
  loss2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels))
  if doWrite:
    varDict2={layer_name+'weights': weights2}
    varDict2[layer_name+'biases'] = biases2
    variable_summaries_list(varDict2)
    tf.scalar_summary('loss/' + layer_name, loss2)    
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer2 = tf.train.GradientDescentOptimizer(0.5).minimize(loss2)
  # Predictions for the training, validation. 
  train_prediction2 = tf.nn.softmax(logits2)
  valid_prediction2 = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights2) + biases2)
  # Predictions for the test data set. 
  test_prediction2 = tf.nn.softmax(tf.matmul(tf_test_dataset, weights2) + biases2)                                    

  # WITHOUT PRIORS REDUCED DATA
  layer_name = 'layer1_normal_reducedDatas'
  weights3 = tf.Variable(weights.initialized_value(),name='weights3')
  biases3 = tf.Variable(biases.initialized_value(),name='biases3')
  # Training computation.
  # Find the classification estimation 'z' value
  logits3 = tf.matmul(tf_train_dataset, weights3) + biases3
  # Calculate the cross-entropy loss. 
  loss3 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits3, tf_train_labels))
  if doWrite:
    varDict3={layer_name+'weights': weights3}
    varDict3[layer_name+'biases'] = biases3
    variable_summaries_list(varDict3)
    tf.scalar_summary('loss/' + layer_name, loss3)    
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer3 = tf.train.GradientDescentOptimizer(0.5).minimize(loss3)
  # Predictions for the training, validation. 
  train_prediction3 = tf.nn.softmax(logits3)
  valid_prediction3 = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights2) + biases3)
  # Predictions for the test data set. 
  test_prediction3 = tf.nn.softmax(tf.matmul(tf_test_dataset, weights3) + biases3)                                    

train_acc_priors = []
valid_acc_priors = []
train_acc_normal = []
valid_acc_normal = []
train_acc_normal_red = []
valid_acc_normal_red = []
num_steps = 5000

train1 = True
train2 = True
train3 = True
with tf.Session(graph=graph) as session:
  # Initialize
  tf.initialize_all_variables().run()    
  summary_op = tf.merge_all_summaries()
  train_dir = 'tmp'
  summary_writer = tf.train.SummaryWriter(train_dir, session.graph)  
  print('Initialized')
  for step in range(num_steps):
    # Run the computations.
    if train1:
      # With priors
      _, lvl1, predictions1 = session.run([optimizer1, loss1, train_prediction1])
    if train2:
      # Without priors
      _, lvl2, predictions2 = session.run([optimizer2, loss2, train_prediction2])
    if train3:
      # Without priors
      _, lvl3, predictions3 = session.run([optimizer3, loss3, train_prediction3])
    
    if (step % 100 == 0):
      if doWrite:
        summary_str = session.run(summary_op)
        summary_writer.add_summary(summary_str, step)  

      acc1 = accuracy(predictions1, train_labels[:train_subset, :])
      acc2 =  accuracy(valid_prediction1.eval(), valid_labels)
      train_acc_priors.append(acc1)
      valid_acc_priors.append(acc2)
      print('With prior. Loss at step %d: %f . Training accuracy: %.1f%% . Validation accuracy: %.1f%%' % (step, lvl1 , acc1, acc2))

      acc1 = accuracy(predictions2, train_labels[:train_subset, :])
      acc2 =  accuracy(valid_prediction2.eval(), valid_labels)
      train_acc_normal.append(acc1)
      valid_acc_normal.append(acc2)
      print('Nope prior. Loss at step %d: %f . Training accuracy: %.1f%% . Validation accuracy: %.1f%%' % (step, lvl2 , acc1, acc2))

      acc1 = accuracy(predictions3, train_labels[:train_subset, :])
      acc2 =  accuracy(valid_prediction3.eval(), valid_labels)
      train_acc_normal_red.append(acc1)
      valid_acc_normal_red.append(acc2)
      print('Nope prior. Loss at step %d: %f . Training accuracy: %.1f%% . Validation accuracy: %.1f%%' % (step, lvl3 , acc1, acc2))
    stopthresh = 0.5;
    if step > 1000:
      if testAccuracy(train_acc_priors):
        train1 = False
        print('No longer training w/ priors')
      if testAccuracy(train_acc_normal):
        train2 = False
        print('No longer training w/ normal method')
      if testAccuracy(train_acc_normal):
        train3 = False
        print('No longer training w/ normal + reduced data')
    if train1==False & train2==False & train3==False:
      print('Stopping Training at step %d', step)
      break;
  print('Test accuracy with prior: %.1f%%' % accuracy(test_prediction1.eval(), test_labels))
  print('Test accuracy no prior: %.1f%%' % accuracy(test_prediction2.eval(), test_labels))
  print('Test accuracy no prior on small data: %.1f%%' % accuracy(test_prediction3.eval(), test_labels))
'''

